In [ ]:
import speech_recognition as sr
import pyttsx3
import wikipedia
import datetime
import requests

# === Setup ===
engine = pyttsx3.init()

def speak(text):
    print("Assistant:", text)
    engine.say(text)
    engine.runAndWait()

def listen():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    try:
        command = recognizer.recognize_google(audio)
        print("You said:", command)
        return command.lower()
    except sr.UnknownValueError:
        speak("Sorry, I didn't understand that.")
        return ""
    except sr.RequestError:
        speak("Speech service is unavailable.")
        return ""

# === Date and Time ===
def get_time():
    now = datetime.datetime.now()
    return now.strftime("%I:%M %p")

def get_date():
    now = datetime.datetime.now()
    return now.strftime("%B %d, %Y")

# === Weather ===
def get_weather(city):
    api_key = "55c514f6756a4bdb077d0bb28aefa2a8"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    try:
        response = requests.get(url).json()
        if response.get("cod") != 200:
            return "I couldn't find the weather for that city."
        desc = response["weather"][0]["description"]
        temp = response["main"]["temp"]
        return f"The weather in {city} is {desc} with a temperature of {temp} degrees Celsius."
    except:
        return "Failed to get weather information."

# === Handle Voice Command ===
def handle_command(command):
    if "hello" in command:
        speak("Hello! How can I help you today?")
    elif "time" in command:
        current_time = get_time()
        speak(f"The current time is {current_time}")
    elif "date" in command:
        current_date = get_date()
        speak(f"Today's date is {current_date}")
    elif "weather" in command:
        speak("Please tell me the city name.")
        city = listen()
        if city:
            weather_info = get_weather(city)
            speak(weather_info)
        else:
            speak("I didn't catch the city name.")
    elif "who is" in command or "what is" in command or "tell me about" in command:
        query = command.replace("who is", "").replace("what is", "").replace("tell me about", "").strip()
        try:
            summary = wikipedia.summary(query, sentences=2)
            speak(summary)
        except Exception:
            speak("Sorry, I couldn't find information on that.")
    elif "exit" in command or "quit" in command or "stop" in command:
        speak("Goodbye!")
        return "exit"
    else:
        speak("I don't understand that command.")

# === Main Loop ===
def main():
    speak("Hi! I am your voice assistant.")
    while True:
        command = listen()
        if command:
            if handle_command(command) == "exit":
                break

if __name__ == "__main__":
    main()
